# How to findout luxury brands
- 명품의 조건 / 기준
  1. Scarity & Price
  2. Brand history
  3. Product quality

*The Strategist blog([link](http://soob.me/publy-project/)), PUBLY magazine([link](https://publy.co/set/170))*

## Preprocessing

In [1]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     | 481kB 9.3MB/s
  Running setup.py bdist_wheel for tqdm ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-w70sex7r/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook as tqdm
import json
from google.colab import drive 
import urllib.request
from pandas.io.json import json_normalize
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
prod = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Pruduct.csv')
mast = pd.read_csv('gdrive/My Drive/공모전/Lpoint/data/Master.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### 자료형 변환
- **object** to **integer**

In [6]:
prod['PD_BUY_AM'] = pd.to_numeric(prod['PD_BUY_AM'].str.replace(',',''))
prod['PD_BUY_CT'][29649] = 1440
prod['PD_BUY_CT'][34105] = 1387
prod['PD_BUY_CT'] = pd.to_numeric(prod['PD_BUY_CT'])

# sess['TOT_SESS_HR_V'] = pd.to_numeric(sess['TOT_SESS_HR_V'].str.replace(',',''))
# sess['SESS_HR_PAG_VIEW'] = sess['TOT_SESS_HR_V'] / sess['TOT_PAG_VIEW_CT']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### 브랜드명 수정
- 브랜드명에서 **불필요한 단어** 제거(예. 향수, 괄호, 공백 등)
- **잘못입력된** 브랜드명 수정

In [0]:
prod['PD_BRA_NM'] = prod['PD_BRA_NM'].map(lambda x: x.split('(')[0].replace('[','').replace(']','').replace(' ',''))
prod.loc[prod['PD_BRA_NM']=='MADE IN ITALY','PD_BRA_NM'] = 'V73'
prod.loc[prod['PD_BRA_NM'].isin(['1909535','19330640825']),'PD_BRA_NM'] = 'KL'
prod.loc[prod['PD_BRA_NM']=='19386640906','PD_BRA_NM'] = '지이크'
prod.loc[prod['PD_BRA_NM'].isin(['1907517','1924113','1906469','1906471','1902295','1908178','1881326','1924116','1906492','1902287']),'PD_BRA_NM'] = 'CC collect'
prod.loc[prod['PD_BRA_NM'].isin(['19379580904','19364580830','1872892','1898551','18703070925','19359410830']),'PD_BRA_NM'] = '더아이잗'
prod.loc[prod['PD_BRA_NM'].isin(['1904418','1896342']),'PD_BRA_NM'] = '보니스팍스'
prod.loc[prod['PD_BRA_NM'].isin(['1912506','1912518']),'PD_BRA_NM'] = '리스트'
prod.loc[prod['PD_BRA_NM']=='19326280823','PD_BRA_NM'] = 'ENC'
prod.loc[prod['PD_BRA_NM'].isin(['19266920905','19266920905','19266920905']),'PD_BRA_NM'] = '요하넥스'
prod.loc[prod['PD_BRA_NM']=='1913802','PD_BRA_NM'] = '꼼빠니아'
prod.loc[prod['PD_BRA_NM']=='1917759','PD_BRA_NM'] = '인디안'

In [0]:
prod_mast = pd.merge(prod, mast, how = 'inner', on=['PD_C'])

# df[df['PD_BUY_CT']>=150000].sort_values(by=['PD_BUY_AM'],ascending=False)
# cate_list= ['패션잡화','여성의류','화장품/뷰티케어','남성의류']
cate_list= ['패션잡화','여성의류','유아동의류','스포츠패션','남성의류']
prod_mast1 = prod_mast[prod_mast['CLAC1_NM'].isin(cate_list)]

In [0]:
l = ['18703070925','1872892','1881326','1896342','1898551','1902287','1902295','1904418','1906469','1906471','1906492','1907517','1908178',
     '1912506','1912518','1913802', '1917759','1924113','1924116','19266920905','19326280823','19359410830','19364580830','19379580904']
prod_mast1[prod_mast1['PD_BRA_NM'].isin(l)]

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM


## Sacrity & Price


### By Price
- 가격을 통해 명품을 판별하자!!
(카테고리별로 평균가격이 상위 25% 이상인 브랜드)

In [0]:
df = prod_mast1.groupby(['CLAC1_NM','PD_BRA_NM']).agg({'PD_BUY_AM':'mean','PD_BUY_CT':'sum'}).reset_index()
df['PD_BUY_AM'] = round(df['PD_BUY_AM'])

In [0]:
df_c = df[df['CLAC1_NM'].isin(['여성의류','남성의류'])].groupby(['PD_BRA_NM']).agg({'PD_BUY_AM':'mean','PD_BUY_CT':'sum'}).sort_values(by=['PD_BUY_AM'], ascending=False).reset_index()
df_f = df[df['CLAC1_NM']=='패션잡화'].sort_values(by=['PD_BUY_AM'], ascending=False).reset_index()[['PD_BRA_NM','PD_BUY_AM','PD_BUY_CT']]
df_b = df[df['CLAC1_NM']=='화장품/뷰티케어'].sort_values(by=['PD_BUY_AM'], ascending=False).reset_index()[['PD_BRA_NM','PD_BUY_AM','PD_BUY_CT']]

brn_c_price = list(df_c[df_c['PD_BUY_AM']>=df_c['PD_BUY_AM'].quantile(0.75)]['PD_BRA_NM'])
brn_f_price = list(df_f[df_f['PD_BUY_AM']>=df_f['PD_BUY_AM'].quantile(0.75)]['PD_BRA_NM'])
brn_b_price = list(df_b[df_b['PD_BUY_AM']>=df_b['PD_BUY_AM'].quantile(0.75)]['PD_BRA_NM'])

In [19]:
brand_price = brn_c_price
brand_price.extend(brn_f_price)
brand_price.extend(brn_b_price)
brand_price = list(set(brand_price))
len(brand_price)

851

In [20]:
brand_price

['유랑스',
 '인터메조',
 '폴리폴리',
 '조르쥬레쉬',
 '조이그라이슨',
 '슈즈브루니',
 '립밤',
 '라메르',
 '타마',
 'OLIVIAHASSLER',
 '폴로랄프로렌',
 '후라밍고',
 '브리스톤',
 '밀라숀',
 '레베카밍코프',
 '존스톤',
 '알레그리',
 '그레이',
 '슈베이스',
 '이웨스트',
 '디아서',
 '베카앤벨',
 '태그호이어',
 '엠포리오알마니',
 '라뉴',
 '디스퀘어드',
 '파슬',
 '디블리스',
 '호간',
 '틸버리',
 '토마스브라운',
 '몽클레르',
 '로보',
 '시리즈',
 '아스페시',
 '사스',
 '닐바렛',
 '에르메스',
 '모이모이',
 '슈퍼콤마비',
 '론칸토',
 '클루드클레어',
 '스텔라맥카트니',
 '기비',
 '트리샤주얼리',
 '마나스',
 '데이트',
 '론카토',
 '몰리즈디럭스',
 '파르베',
 '노멀',
 '제옥스',
 '로터프',
 '엘비엘',
 '갤럭시',
 '브룩스브라더스레드플리스',
 '이세이미야케',
 '에끌라떼',
 '울프_ACC',
 'AlvieroMartini1AClasse',
 '에드하디',
 '모어올레스',
 '캐리스노트',
 '미아백',
 '자스페로',
 '클라쎄일사',
 '시슬리프리미엄',
 '워너원',
 '처치스',
 '비앤비골드',
 '루키버드',
 '브루노말리',
 '아르마니주니어',
 '페라가모.',
 '디스퀘어드2',
 '베네통프리미엄',
 '짐리키',
 '피에르아르디',
 '본사직영정품',
 '에피그램',
 '까스따네르',
 '키이스',
 '세컨플로어',
 '구카',
 '3.1필립림',
 '체사레파치오티',
 '로사케이',
 '휴고보스',
 '스피도미터오피셜',
 '워크센스',
 '오보에다모르',
 '제이청',
 '융한스',
 '아쿠티스1942',
 '크롬베즈',
 '라크루아상',
 '벤크',
 '알렉산더맥퀸',
 '존폴쥬얼리',
 '바이네르',
 '에테르노',
 '에비뉴67',
 'PripebyMisope',
 '빈폴

### By Sacrity
- 같은 브랜드를 다른 쇼핑몰에서 팔지 않는다면 희소성있는 브랜드?

## Product Quality
- 상품의 질을 객과적으로 판단하고 수치화하기 어렵다;;;;;;;;

### By Quality
- 인지도/평판을 통해 대체할 수 있지 않을까?
- 가격이 높은 브랜드들의 인지도를 알아보자!!

In [0]:
client_id = ""
client_secret = ""
url = "https://openapi.naver.com/v1/datalab/search"

In [0]:
bra_list = []
for brand in brand_price:
  bra_list.append({"groupName": brand ,"keywords": [brand]})
bra_list[0], bra_list[4] = bra_list[4], bra_list[0]

In [26]:
bra_trend = []
lists = [bra_list[0]]
for i in tqdm(range(1,len(bra_list[:]),4)):
  b_trends = []
  lists.extend(bra_list[i:i+4])
  body = '{\"startDate\":\"2016-01-01\",\"endDate\":\"2019-01-30\",\"timeUnit\":\"month\",\"keywordGroups\":%s}'%json.dumps(lists)
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  request.add_header("Content-Type","application/json")
  response = urllib.request.urlopen(request, data=body.encode("utf-8"))
  rescode = response.getcode()
  response_body = response.read().decode('utf-8')
  trends = json.loads(response_body)['results']
  for t in trends:
    b_trends.append({
        'title':t['title'],
        'ratio':np.mean([d['ratio'] for d in t['data']])
    })
  bra_trend.extend(b_trends)
  max_b =  max(b_trends, key=lambda x:x['ratio'])
  lists = list(filter(lambda x:x['groupName']==max_b['title'], lists))

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [0]:
brand_mp = json_normalize(bra_trend).drop_duplicates().reset_index()

In [0]:
brand_mp = brand_mp[brand_mp['ratio']>=brand_mp['ratio'].quantile(0.75)]

#### 명품리스트
- 추출한 결과물을 보니 썩 만족스럽진 않다.
- 롯데의 리스트를 쓰는 것이 나을 것 같다.
- 그래도 혹시 모르니 일단 저장


In [0]:
brand_mp[['title','ratio']].to_csv('gdrive/My Drive/공모전/Lpoint/명품 골라내기/my_luxury_list.csv',index=False)

## Brand History
- 브랜드의 역사를 알아내기 힘들다...

### By Brand
- 롯데에서는 어떤 브랜드를 명품으로 선정했을까?
- 이 정보를 참고하자!!!

In [0]:
url_csm = 'http://www.lotte.com/display/searchBrandInitialReNew.lotte?upr_disp_no=1692074&kor_eng_sct_cd=&sch_nm=&sch_nm_fortis='

res = requests.get(url_csm).text
soup = bs(res, 'html.parser').find('div',{'class':'group'})
words_csm = []
for alpha in soup.find_all('li'):
  for word in alpha.find_all('li'):
    words_csm.extend(word.text.replace(')','').replace(' ','').split('('))
print(words_csm)

['겔랑', '꼬달리', '끌레드뽀보떼', '니콜라이', '닐스야드', '닥터자르트', '달팡', '더사가오브수', '더센트오프디파쳐', '데코르테', '델라롬', '디어달리아', '딥티크', '라메르', '라보라토리오올파티보', '라비오뜨', '라프레리', '랑방', '랑콤', '랩시리즈', '로라메르시에', '로베르트피게', '록시땅', '르꽁뚜아르뒤뱅', '르네휘테르', '리리코스', '리엔케이', '메이크업포에버', '메종프란시스커정', '멜비타', '몰튼브라운', '무코타', '미우미우', '미키모토코스메틱', '바닐라코', '바비브라운', '바이레도', '베네피트', '벨포트', '부르조아', '불가리', '블리스', '비디비치', '비오템', '빌리프', '산심', '산타마리아노벨라', '샤넬', '샹테카이', '설화수', '세르주루텐', '숨37˚', '슈에무라', '스틸라', '시세이도', '시슬리', '시에로코스메틱', '쌍빠', '씨앤피알엑스', '아닉구딸', '아모레퍼시픽', '아베다', '아이뽀', '아쿠아디파르마', '아틀리에코롱', '안나수이', '어딕션', '어반디케이', '에르바리오토스카노', '에르보리앙', '에스쁘아', '에스티로더', '에어린', '엘리자베스아덴', '오리진스', '오휘', '윈뉘어발리', '이금희피부밥', '이솝', '이오시카', '입생로랑', '정샘물뷰티', '조말론런던', 'JoMaloneLondon', '조르지오아르마니', '주디스리버', '쥴라이', '쥴리크', '질스튜어트', '향수', '츄파춥스뷰티', '크리니크', '크리드', '클라란스', '클라리소닉', '키엘', '탄', '토리버치', '톰포드', '투쿨포스쿨', '트리아뷰티', '펜할리곤스', '폴앤조', '프라나롬', '프라다', '프레데릭말', '프레쉬', '프리메라', '필로소피', '하이코스퍼퓸', '헤라', '헤어리추얼바이시슬리', '후', 'CEOParfums', 'DIOR', 'DIORBACKSTAGE', 'M.A.C', 'N

In [0]:
url_lux = 'http://www.lotte.com/display/searchBrandInitialReNew.lotte?upr_disp_no=5407978&kor_eng_sct_cd=&sch_nm=&sch_nm_fortis='

res = requests.get(url_lux).text
soup = bs(res, 'html.parser').find('div',{'class':'group'})
words_lux = []
for alpha in soup.find_all('li'):
  for word in alpha.find_all('li'):
    words_lux.append(word.text.split('(')[0].replace(' ',''))
print(words_lux)

['게스', '겐조', '노미네이션', '닥스', '더블엠', '라도', '러브캣', '러브캣비쥬', '레노마', '로이드', '로즈몽', '로즈몽', '루이까또즈', '루이까또즈', '루첸리', '리타모니카', '마리아꾸르끼', '마리앤잭', '마이클코어스', '마크바이마크제이콥스', '메트로시티', '메트로시티', '메트로시티', '몬데인', '몰리즈', '미니골드', '미쉘에블랑', '바쵸바치', '베이비지', '뷰', '브룩스브라더스', '비너스', '비비안', '비비안웨스트우드', '비체', '샤링', '순토', '스와로브스키', '스와치', '스타일러스', '스톤헨지', '시티즌', '아가타', '아가타', '아르키메데스', '아이그너', '아이그너', '아쿠아스큐텀', '에스티듀퐁', '엘르', '엠포리오아르마니', '올리비아버튼', '융한스', '제이에스티나', '지샥', '질스튜어트', '캘빈클라인', '캘빈클라인', '킨트', '토마스사보', '토마스사보', '티르리르', '티쏘', '페라가모', '프레드릭콘스탄트', '프린세스', '해밀턴', '휴고보스', 'DKNY', '아가타', '아가타', '아르키메데스', '아이그너', '아이그너', '아쿠아스큐텀', '바쵸바치', '베이비지', '브룩스브라더스', '비체', '시티즌', '캘빈클라인', '캘빈클라인', '닥스', '더블엠', 'DKNY', '엘르', '엠포리오아르마니', '페라가모', '프레드릭콘스탄트', '게스', '지샥', '해밀턴', '휴고보스', '메트로시티', '융한스', '제이에스티나', '질스튜어트', '겐조', '킨트', '러브캣', '러브캣비쥬', '루이까또즈', '루첸리', '마리아꾸르끼', '마리앤잭', '마이클코어스', '마크바이마크제이콥스', '메트로시티', '몬데인', '몰리즈', '미니골드', '미쉘에블랑', '노미네이션', '올리비아버튼', '프린세스', '라도', '레노마', '로즈몽', '로즈몽', '리타모니카', '샤링', '순토', '스와로브스키'

In [0]:
url_clc = 'http://www.lotte.com/display/searchBrandInitialReNew.lotte?upr_disp_no=5559034&kor_eng_sct_cd=&sch_nm=&sch_nm_fortis='

res = requests.get(url_clc).text
soup = bs(res, 'html.parser').find('div',{'class':'group'})
words_clc = []
for alpha in soup.find_all('li'):
  for word in alpha.find_all('li'):
    words_clc.extend(word.text.replace(')','').replace(' ','').split('('))
print(words_clc)

['겐조', 'KENZO', '골든구스', 'GOLDENGOOSE', '꼼데가르송', 'CommedesGarcons', '끌로에', 'CHLOE', '노비스', 'NOBIS', '닐바렛', 'neilbarrett', '듀베티카', 'duvetica', '디스퀘어드2', 'DSQUARED2', '레페토', 'Lepetto', '릭오웬스', 'RICKOWENS', '마르니', 'MARNI', '마크제이콥스', 'MarcJacobs', '막스마라', 'MaxMara', '멀버리', 'Mulberry', '몽블랑', 'MONTBLANC', '몽클레어', 'Moncler', '무스너클', 'MOOSEKNUCKLES', '미우미우', 'MIUMIU', '발렌시아가', 'BALENCIAGA', '발렌티노', 'VALENTINO', '발망', 'BALMAIN', '보테가베네타', 'BOTTEGAVENETA', '생로랑', 'SAINTLAURENT', '셀린느', 'CELINE', '스텔라메카트니', 'StellaMcCartney', '스톤아일랜드', 'STONEISLAND', '스튜어트와이츠먼', 'StuartWeitzman', '아크네', 'Acne', '안야힌드마치', 'ANYAHINDMARCH', '알렉산더맥퀸', 'ALEXANDERMCQUEEN', '알렉산더왕', 'ALEXANDERWANG', '에르노', 'herno', '에트로', 'ETRO', '올라카일리', 'OrlaKiely', '이자벨마랑', 'ISABELMARANT', '자넬라토', 'ZANELLATO', '지방시', 'GIVENCHY', '커먼프로젝트', 'COMMONPROJECTS', '코치', 'COACH', '키아라페라그니', 'chiaraferragni', '토리버치', 'TORYBURCH', '톰브라운', 'THOMBROWNE', '투미', 'TUMI', '폴스미스', 'Paulsmith', '프라다', 'PRADA', '프로엔자슐러', 'proenzaschouler', '피에르아르디', 'pi

In [0]:
brand_lotte = words_csm
brand_lotte.extend(words_lux)
brand_lotte.extend(words_clc)
brand_lotte = list(set(brand_lotte))
len(brand_lotte)

275

In [0]:
brand_lotte = pd.Series(brand_lotte)

In [0]:
brand_lotte

In [0]:
brand_lotte.to_csv("gdrive/My Drive/공모전/엘포인트/data/lotte_luxury_list.csv",index = False, header = ['BRA_NM'])

## Luxury Brand List
- 혹시 몰라서 만들긴 한다.....
- 두가지 리스트에 공통적으로 등장한 명품 브랜드 리스트 생성
   - 기준에 따라 추출한 명품 리스트
   - 롯데에서 선정한 명품 리스트


In [0]:
lotte_list = set(brand_lotte)
my_list=set(brand_mp['title'])

lux_list = list(lotte_list.intersection(my_list))

In [0]:
len(lux_list)

78

In [0]:
lux_list

['마크제이콥스',
 '생로랑',
 '로즈몽',
 '프레드릭콘스탄트',
 '폴스미스',
 '발망',
 '입생로랑',
 '브룩스브라더스',
 '이자벨마랑',
 '시슬리',
 '알렉산더맥퀸',
 '에트로',
 '코치',
 '발렌티노',
 '클라란스',
 '메트로시티',
 '헤라',
 '에르노',
 '리리코스',
 '바비브라운',
 '랑콤',
 '시티즌',
 '프레쉬',
 '조르지오아르마니',
 '샤넬',
 '겐조',
 '에스티로더',
 '겔랑',
 '달팡',
 '몽블랑',
 '마이클코어스',
 '랩시리즈',
 '제이에스티나',
 '멀버리',
 '이솝',
 '비비안웨스트우드',
 '러브캣',
 '발렌시아가',
 '레페토',
 '커먼프로젝트',
 '지샥',
 '막스마라',
 '산타마리아노벨라',
 '톰포드',
 '투미',
 '보테가베네타',
 '티쏘',
 '랑방',
 '톰브라운',
 '프린세스',
 '토리버치',
 '바이레도',
 '무스너클',
 '리엔케이',
 '미우미우',
 '캘빈클라인',
 '시세이도',
 '프라다',
 '몽클레어',
 '페라가모',
 '키엘',
 '셀린느',
 '끌로에',
 '질스튜어트',
 '오휘',
 '지방시',
 '스톤아일랜드',
 '설화수',
 '릭오웬스',
 '불가리',
 '알렉산더왕',
 '비오템',
 '아크네',
 '순토',
 '꼼데가르송',
 '골든구스',
 '해밀턴',
 '후']

## 부록
- 네이버 API를 이용해서 브랜드 + 중분류 검색어트렌드를 카운트하는 코드
  - group

In [0]:
df1 = prod_mast1.groupby(['CLAC2_NM','CLAC3_NM','PD_BRA_NM']).agg({'PD_BUY_AM':'mean','PD_BUY_CT':'sum'}).reset_index()

In [0]:
df1 = df1[df1['PD_BRA_NM'].isin(brand_lotte)]

In [0]:
bra_list = []
for category in tqdm(df1['CLAC2_NM'].drop_duplicates()):
  for brand in tqdm(df1[df1['CLAC2_NM'] == category]['PD_BRA_NM'].drop_duplicates()):
    a = [brand + category]
    b = [brand + item for item in df1[df1['CLAC2_NM'] == category]['CLAC3_NM'].drop_duplicates()]
    b.extend(a)
    bra_list.append({"groupName": a[0],
                    "keywords": b
                    })



In [0]:
bra_list

In [0]:
bra_trend = []
lists = [bra_list[0]]
for i in tqdm(range(1,len(bra_list[:]),4)):
  b_trends = []
  lists.extend(bra_list[i:i+4])
  body = '{\"startDate\":\"2016-01-01\",\"endDate\":\"2019-01-30\",\"timeUnit\":\"month\",\"keywordGroups\":%s}'%json.dumps(lists)
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  request.add_header("Content-Type","application/json")
  response = urllib.request.urlopen(request, data=body.encode("utf-8"))
  rescode = response.getcode()
  response_body = response.read().decode('utf-8')
  trends = json.loads(response_body)['results']
  for t in trends:
    b_trends.append({
        'title':t['title'],
        'ratio':np.mean([d['ratio'] for d in t['data']])
    })
  bra_trend.extend(b_trends)
  max_b =  max(b_trends, key=lambda x:x['ratio'])
  lists = list(filter(lambda x:x['groupName']==max_b['title'], lists))

{"startDate":"2018-04-01","endDate":"2018-09-30","timeUnit":"date","results":[{"title":"루키버드","keywords":["루키버드 명품","명품 루키버드"],"data":[]},{"title":"닥스레이디스","keywords":["닥스레이디스 명품","명품 닥스레이디스"],"data":[]},{"title":"아이엠지프리미엄","keywords":["아이엠지프리미엄 명품","명품 아이엠지프리미엄"],"data":[]},{"title":"ALBERTO FERMANI","keywords":["ALBERTO FERMANI 명품","명품 ALBERTO FERMANI"],"data":[]},{"title":"알레그리","keywords":["알레그리 명품","명품 알레그리"],"data":[]}]}
